In [20]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [21]:
import mlflow

In [36]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-expe/mlruns/1', creation_time=1747637206697, experiment_id='1', last_update_time=1747637206697, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [23]:
def read_dataframe(filename):
    df = pd.read_parquet(filename).head(90000)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [24]:
df_train =read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-02.parquet')

In [25]:
len(df_train), len(df_val)

(87795, 88147)

In [26]:
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) + '_' + df_val['DOLocationID'].astype(str)

In [27]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [28]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [29]:

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

26.665223468577214

In [30]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [31]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet")
    mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [32]:
import xgboost as xgb

In [33]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [35]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [01:04:57] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.28116                           
[1]	validation-rmse:8.95098                           
[2]	validation-rmse:8.64628                           
[3]	validation-rmse:8.36408                           
[4]	validation-rmse:8.10276                           
[5]	validation-rmse:7.86180                           
[6]	validation-rmse:7.63982                           
[7]	validation-rmse:7.43566                           
[8]	validation-rmse:7.24774                           
[9]	validation-rmse:7.07312                           
[10]	validation-rmse:6.91417                          
[11]	validation-rmse:6.76824                          
[12]	validation-rmse:6.63501                          
[13]	validation-rmse:6.51252                          
[14]	validation-rmse:6.40105                          
[15]	validation-rmse:6.29997                          
[16]	validation-rmse:6.20464                          
[17]	validation-rmse:6.11766                          
[18]	valid

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [01:08:38] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.19985                                                        
[1]	validation-rmse:8.79926                                                        
[2]	validation-rmse:8.43327                                                        
[3]	validation-rmse:8.09998                                                        
[4]	validation-rmse:7.79635                                                        
[5]	validation-rmse:7.52065                                                        
[6]	validation-rmse:7.27046                                                        
[7]	validation-rmse:7.04401                                                        
[8]	validation-rmse:6.83961                                                        
[9]	validation-rmse:6.65435                                                        
[10]	validation-rmse:6.48770                                                       
[11]	validation-rmse:6.33700                                                

...

[0]	validation-rmse:6.15740                                                      
[1]	validation-rmse:5.34220                                                      
[2]	validation-rmse:5.13125                                                      
[3]	validation-rmse:5.07169                                                      
[4]	validation-rmse:5.04253                                                      
[5]	validation-rmse:5.03773                                                      
[6]	validation-rmse:5.03760                                                      
[7]	validation-rmse:5.02809                                                      
[8]	validation-rmse:5.02564                                                      
[9]	validation-rmse:5.02140                                                      
[10]	validation-rmse:5.01885                                                     
[11]	validation-rmse:5.01836                                                     
[12]	validation-

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [01:41:37] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:5.42235                                                      
[1]	validation-rmse:5.10284                                                      
[2]	validation-rmse:5.06229                                                      
[3]	validation-rmse:5.05367                                                      
[4]	validation-rmse:5.04931                                                      
[5]	validation-rmse:5.04002                                                      
[6]	validation-rmse:5.03928                                                      
[7]	validation-rmse:5.03250                                                      
[8]	validation-rmse:5.02818                                                      
[9]	validation-rmse:5.02878                                                      
[10]	validation-rmse:5.02165                                                     
[11]	validation-rmse:5.02082                                                     
[12]	validation-

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [01:41:51] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:5.31938                                                      
[1]	validation-rmse:5.30318                                                      
[2]	validation-rmse:5.31134                                                      
[3]	validation-rmse:5.30913                                                      
[4]	validation-rmse:5.31578                                                      
[5]	validation-rmse:5.30860                                                      
[6]	validation-rmse:5.30662                                                      
[7]	validation-rmse:5.30872                                                      
[8]	validation-rmse:5.31231                                                      
[9]	validation-rmse:5.31279                                                      
[10]	validation-rmse:5.31881                                                     
[11]	validation-rmse:5.32209                                                     
[12]	validation-